# Set up your environment

Let's use a `.env` file and the `dotenv` library. This allows you to keep your API key secure and separated from your code.

Here’s how you can setup to load the OpenAI API key using a `.env` file:

### Step 1: Install Dependencies
Make sure you have `python-dotenv` installed if you're using `.env` files:
```bash
pip install python-dotenv
pip install -U langsmith
pip install openai
pip install langchain_openai
```

### Step 2: Create a `.env` File
In your project directory, create a `.env` file with the following content:
```
OPENAI_API_KEY=your-api-key-here
```

### Step 3: Test

In [ ]:
import sys
print(sys.executable)

In [ ]:
%%capture --no-stderr
%pip install --quiet -U langgraph openai langchain_openai google-generativeai langchain_anthropic

In [ ]:
#updated

import os
import re
import requests
from dotenv import load_dotenv
import google.generativeai as genai
from openai import OpenAI  # ✅ Grok now uses OpenAI-compatible SDK
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from anthropic import Anthropic
from langchain_core.messages import HumanMessage  
from IPython.display import display, Markdown  

# Load environment variables
load_dotenv()

# Function to load API keys safely
def get_env_var(var: str):
    value = os.getenv(var)
    if value is None:
        raise ValueError(f"{var} not found in environment variables. Make sure it is set in your .env file.")
    return value

# Load API keys
openai_api_key = get_env_var("OPENAI_API_COURSE_KEY")  
gemini_api_key = get_env_var("GEMINI_API_KEY")  
anthropic_api_key = get_env_var("ANTHROPIC_API_KEY")  
xai_api_key = get_env_var("XAI_API_KEY")  # ✅ Updated for Grok

# Initialize OpenAI GPT models
gpt4o_chat = ChatOpenAI(model="gpt-4o", temperature=0, openai_api_key=openai_api_key)
gpt4o_mini_chat = ChatOpenAI(model="gpt-4o-mini", temperature=0, openai_api_key=openai_api_key)

# Initialize Claude model
claude_chat = ChatAnthropic(model="claude-3-5-sonnet-20241022", temperature=0, anthropic_api_key=anthropic_api_key)

# Initialize Google Gemini API Client
genai.configure(api_key=gemini_api_key)  
gemini_model = genai.GenerativeModel("gemini-2.0-flash")

# Initialize Grok (XAI API) Client
grok_client = OpenAI(api_key=xai_api_key, base_url="https://api.x.ai/v1")  # ✅ New Grok client

# Create outputs directory if it doesn't exist
os.makedirs("outputs", exist_ok=True)

# Function to generate a safe filename from query text
def sanitize_filename(query: str):
    return re.sub(r'[^\w\s-]', '', query).strip().replace(' ', '_')[:50] + ".md"

# ======================================
# Grok API Integration (Updated)
# ======================================

def query_grok(prompt: str):
    """
    Query the latest Grok model using OpenAI-compatible SDK.
    """
    try:
        completion = grok_client.chat.completions.create(
            model="grok-2-latest",
            messages=[
                {"role": "system", "content": "You are Grok, a chatbot inspired by the Hitchhiker's Guide to the Galaxy."},
                {"role": "user", "content": prompt}
            ],
        )
        return completion.choices[0].message.content
    except Exception as e:
        return f"Error querying Grok: {e}"


# ======================================
# Unified Response Comparison and Saving
# ======================================

def format_response(response: str) -> str:
    """
    Standardize the formatting of the model's response for consistent readability.
    """
    sections = response.split("\n")
    formatted_response = []
    for section in sections:
        if section.strip().endswith(":"):
            formatted_response.append(f"\n### {section.strip()}")  # Markdown headers
        else:
            formatted_response.append(section.strip())
    return "\n".join(formatted_response)

def save_comparison_to_markdown(prompt: str, results: dict, filename: str):
    """
    Save the formatted output of model comparisons to a Markdown file.
    """
    try:
        with open(filename, "w", encoding="utf-8") as f:
            f.write(f"# Prompt:\n\n{prompt}\n\n")
            f.write("=" * 80 + "\n\n")
            
            for model, response in results.items():
                f.write(f"## {model} Response\n\n")
                f.write(f"{response}\n\n")
                f.write("-" * 80 + "\n\n")
        
        print(f"✅ Output saved to: {filename}")
    except Exception as e:
        print(f"❌ Error saving to {filename}: {e}")

def compare_responses(prompt: str, save_dir="outputs"):
    """
    Compare responses from GPT-4o, GPT-4o-mini, Claude-3.5, Gemini-2.0, and Grok models for the same prompt.
    Save to a Markdown file and display in Jupyter Notebook.
    """
    results = {}

    # GPT-4o
    gpt4_response = gpt4o_chat.invoke([HumanMessage(content=prompt)]).content
    results["GPT-4o"] = format_response(gpt4_response)

    # GPT-4o-mini
    gpt4o_mini_response = gpt4o_mini_chat.invoke([HumanMessage(content=prompt)]).content
    results["GPT-4o-mini"] = format_response(gpt4o_mini_response)

    # Claude-3.5
    claude_response = claude_chat.invoke(prompt).content
    results["Claude-3.5-Sonnet"] = format_response(claude_response)

    # Gemini-2.0 Flash
    gemini_response = gemini_model.generate_content(prompt).text
    results["Gemini-2.0-Flash"] = format_response(gemini_response)

    # Grok-2-Latest
    grok_response = query_grok(prompt)
    results["Grok-2-Latest"] = format_response(grok_response)

    # Ensure output directory exists
    os.makedirs(save_dir, exist_ok=True)

    # Generate the output filename based on the query
    output_filename = os.path.join(save_dir, sanitize_filename(prompt))

    # Save comparison results
    save_comparison_to_markdown(prompt, results, output_filename)

    # Display formatted results in Jupyter Notebook
    output_markdown = f"""# Query: {prompt}

## GPT-4o Response:
{results["GPT-4o"]}

## GPT-4o-mini Response:
{results["GPT-4o-mini"]}

## Claude-3.5-Sonnet Response:
{results["Claude-3.5-Sonnet"]}

## Gemini-2.0-Flash Response:
{results["Gemini-2.0-Flash"]}

## Grok-2-Latest Response:
{results["Grok-2-Latest"]}
"""

    display(Markdown(output_markdown))

# ======================================
# Run the Comparison
# ======================================

query_text = "Explain how AI works"
compare_responses(query_text)


In [ ]:
from langchain_core.messages import HumanMessage

# Create a message
msg = HumanMessage(content="Hello world", name="Joseph")

# Message list
messages = [msg]

# Invoke the model with a list of messages 
gpt4o_chat.invoke(messages)

In [ ]:
gpt4o_chat.invoke("hello world")

In [ ]:
gpt4o_mini_chat.invoke("hello world")

In [ ]:
# Set up Tavily using the API key from the environment
tavily_api_key = get_env_var("TAVILY_API_KEY")
from langchain_community.tools.tavily_search import TavilySearchResults
tavily_search = TavilySearchResults(max_results=3, api_key=tavily_api_key)

In [ ]:
# Perform a search with Tavily
search_docs = tavily_search.invoke("What is LangGraph?")
search_docs

In [ ]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.callbacks import get_openai_callback

# Load environment variables from .env file
load_dotenv()

# Function to load environment variables or raise an error if not found
def get_env_var(var: str):
    value = os.getenv(var)
    if value is None:
        raise ValueError(f"{var} not found in environment variables. Make sure it is set in your .env file.")
    return value

# Load API keys from the environment
openai_api_key = get_env_var("OPENAI_API_COURSE_KEY")

# Initialize the OpenAI LLM via LangChain
# gpt_chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, openai_api_key=openai_api_key)
gpt4o_mini_chat = ChatOpenAI(model="gpt-4o-mini", temperature=0, openai_api_key=openai_api_key)

# Define a simple prompt template for conversation
prompt_template = PromptTemplate(
    input_variables=["user_input"],
    template="You are a helpful assistant. User asks: {user_input}. How would you respond?"
)

# Instead of using RunnableSequence, directly chain the prompt and model using the | operator
# conversation_chain = prompt_template | gpt_chat
conversation_chain = prompt_template | gpt4o_mini_chat

try:
    # Start callback for tracing OpenAI usage
    with get_openai_callback() as cb:
        print("LangChain tracing started successfully.")

        # Example conversation - LangChain will trace this operation
        user_input = "What are the benefits of using transformers in natural language processing?"
        response = conversation_chain.invoke({"user_input": user_input})

        print("Agent's Response:")
        print(response)

        # You can extend this conversation further
        follow_up = "Can you explain the difference between GPT and BERT?"
        follow_up_response = conversation_chain.invoke({"user_input": follow_up})

        print("Follow-up Response:")
        print(follow_up_response)

        # Output callback information
        print(f"\nTotal Tokens Used: {cb.total_tokens}")
        print(f"Total Cost (USD): ${cb.total_cost}")

    print("LangChain tracing ended successfully.")

except Exception as e:
    print(f"Error with LangChain tracing: {e}")


In [ ]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.callbacks import get_openai_callback

# Load environment variables from .env file
load_dotenv()

# Function to load environment variables or raise an error if not found
def get_env_var(var: str):
    value = os.getenv(var)
    if value is None:
        raise ValueError(f"{var} not found in environment variables. Make sure it is set in your .env file.")
    return value

# Load API keys from the environment
openai_api_key = get_env_var("OPENAI_API_COURSE_KEY")

# Initialize the OpenAI LLM via LangChain with GPT-4o-mini
gpt4o_mini_chat = ChatOpenAI(model="gpt-4o-mini", temperature=0, openai_api_key=openai_api_key)

# Define a simple prompt template for conversation
prompt_template = PromptTemplate(
    input_variables=["user_input"],
    template="You are a helpful assistant. User asks: {user_input}. How would you respond?"
)

# Chain the prompt and model using the | operator
conversation_chain = prompt_template | gpt4o_mini_chat

try:
    # Start callback for tracing OpenAI usage
    with get_openai_callback() as cb:
        print("LangChain tracing started successfully.")

        # Example conversation - LangChain will trace this operation
        user_input = "What are the benefits of using transformers in natural language processing?"
        response = conversation_chain.invoke({"user_input": user_input})

        print("Agent's Response:")
        print(response)

        # Extend the conversation further
        follow_up = "Can you explain the difference between GPT and BERT?"
        follow_up_response = conversation_chain.invoke({"user_input": follow_up})

        print("Follow-up Response:")
        print(follow_up_response)

        # Output callback information
        print(f"\nTotal Tokens Used: {cb.total_tokens}")
        print(f"Total Cost (USD): ${cb.total_cost}")

    print("LangChain tracing ended successfully.")

except Exception as e:
    print(f"Error with LangChain tracing: {e}")


In [ ]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

# Load environment variables from .env file
load_dotenv()

# Function to retrieve environment variables
def get_env_var(var: str):
    value = os.getenv(var)
    if value is None:
        raise ValueError(f"{var} not found in environment variables. Ensure it is set in your .env file.")
    return value

# Load the OpenAI API key from .env
openai_api_key = get_env_var("OPENAI_API_COURSE_KEY")

# Initialize the OpenAI model with LangChain
llm = ChatOpenAI(model="gpt-4o-mini", openai_api_key=openai_api_key)

# Define a custom prompt template
template = PromptTemplate(
    template="""
    You are a cockney fruit and vegetable seller.
    Your role is to assist your customer with their fruit and vegetable needs.
    Respond using cockney rhyming slang.

    Tell me about the following fruit: {fruit}
    """,
    input_variables=["fruit"]
)

# Format the template with a specific fruit
formatted_prompt = template.format(fruit="apple")

# Invoke the LLM with the formatted prompt
response = llm.invoke([{"role": "user", "content": formatted_prompt}])

# Print the response from the LLM
print("Cockney Seller's Response:", response.content)


In [ ]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

# Load environment variables from .env file
load_dotenv()

# Function to retrieve environment variables or raise an error if missing
def get_env_var(var: str):
    value = os.getenv(var)
    if value is None:
        raise ValueError(f"{var} not found in environment variables. Ensure it is set in your .env file.")
    return value

# Load the OpenAI API key from the environment
openai_api_key = get_env_var("OPENAI_API_COURSE_KEY")

# Initialize the OpenAI LLM using GPT-4o-mini
llm = ChatOpenAI(model="gpt-4o-mini", openai_api_key=openai_api_key)

# Define a reusable prompt template
template = PromptTemplate.from_template("""
You are a cockney fruit and vegetable seller.
Your role is to assist your customer with their fruit and vegetable needs.
Respond using cockney rhyming slang.

Tell me about the following fruit: {fruit}
""")

# Note how we combined the prompt and LLM into a reusable chain
llm_chain = template | llm

# Function to invoke the chain with a given fruit : You invoke the llm_chain passing the template parameters as a dictionary.
def get_cockney_response(fruit: str):
    response = llm_chain.invoke({"fruit": fruit})
    return response.content

# Example usage
if __name__ == "__main__":
    fruit = "apple"
    response = get_cockney_response(fruit)
    print(f"Cockney Seller's Response for '{fruit}':\n{response}")
